In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pyqubo import Binary
from dwave.system.composites import EmbeddingComposite
from dwave.system.samplers import DWaveSampler
import dimod

# 乱数シードを設定
seed_value = 91
np.random.seed(seed_value)

# 量子ビットをnつ用意する
players = 40
interaction_range = players  # 前後n列のプレイヤーと相互作用を持つ

qbits = [Binary(f'q{i:03}') for i in range(players)]

# 個人効用を追加
individual_utilities = []
H = 0
for i in range(players):
    individual_utility = 2 * np.random.rand() - 1  # -1から1の範囲で生成
    individual_utilities.append(individual_utility)
    H += individual_utility * qbits[i]

# 相互作用を追加（前方のみ）
interaction_weights = []
for i in range(players):
    for j in range(1, interaction_range + 1):
        if i + j < players:
            interaction_weight = 2 * np.random.rand() - 1  # -1から1の範囲で生成
            interaction_weights.append((i, j, interaction_weight))
            H += interaction_weight * (qbits[i] == qbits[i + j])

# コンパイル
model = H.compile()
qubo, offset = model.to_qubo()
print('qubo\n{}'.format(qubo))

# D-waveの設定
sampler = EmbeddingComposite(DWaveSampler(endpoint='https://cloud.dwavesys.com/sapi',
                                          token='yourtoken',
                                          solver='Advantage_system6.4'))

num_reads_list = [100, 200, 400]
colors = ['blue', 'green', 'purple', 'orange']
labels = [f'num_reads={num_reads}' for num_reads in num_reads_list]

plt.figure(figsize=(20, 12))

responses = []

for num_reads, color, label in zip(num_reads_list, colors, labels):
    # サンプリングしまくる
    response = sampler.sample_qubo(qubo, num_reads=num_reads)
    responses.append(response)

    # レスポンスの確認
    energies = []
    occurrences = []

    for (sample, energy, occurrence, _) in response.data():
        print('Sample:{} Energy:{} Occurrence:{}'.format(list(sample.values()), energy, occurrence))
        energies.append(energy)
        occurrences.append(occurrence / num_reads)  # 回数の割合に変更

    # グラフを描画
    plt.plot(energies, occurrences, color=color, linestyle='-', label=label)
    plt.scatter(energies, occurrences, c=color, marker='o')  # 点を追加



plt.xlabel('Energy')
plt.ylabel('Occurrence Ratio')
plt.title('Energy vs Occurrence Ratio')
plt.legend()
plt.grid(True)

# ファイル名を設定して保存
file_name = f'result_combined_{min(num_reads_list)}_{max(num_reads_list)}_players{players}_interaction{interaction_range}_seed{seed_value}.png'
plt.savefig(file_name)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations

# 特定のnum_readsのレスポンスデータを使用
response = responses[0]  # 例えば、最初のnum_reads=100のデータを使用

# レスポンスのデータをエネルギーの低い順でソート
sorted_response = sorted(response.data(), key=lambda x: x[1])

# n番目までの解を対象とする
n = 10  # 任意に指定
energy_differences = []

for num in range(n):
    # 出現回数の多い解のエネルギーを取得
    most_frequent_energy = sorted_response[num].energy
    print(f'Energy of {num+1}th most frequent sample: {most_frequent_energy}')

    # 出現回数の多い解を取得
    most_frequent_sample = sorted_response[num].sample

    # 異なる値をとる量子の数ごとにエネルギーを格納する辞書
    all_energies_by_diff_count = {}

    # 出現回数の多い解のサンプルと異なる値をとる量子が1つのすべての組み合わせを生成
    for diff_count in range(1, 2):  # 1つの異なる量子ビットのみを考慮
        for indices in combinations(range(len(most_frequent_sample)), diff_count):
            sample = most_frequent_sample.copy()
            for i in indices:
                sample[f'q{i:03}'] = 1 - sample[f'q{i:03}']  # フリップ
            energy = model.energy(sample, vartype='BINARY')
            
            if diff_count not in all_energies_by_diff_count:
                all_energies_by_diff_count[diff_count] = []
            
            all_energies_by_diff_count[diff_count].append(energy)

    # 最もエネルギーが低い値を取得
    min_energy = min(all_energies_by_diff_count[1])
    print(f'Minimum energy with 1 differing qubit: {min_energy}')

    # エネルギーの差異を計算
    energy_difference = most_frequent_energy - min_energy
    energy_differences.append(energy_difference)

# グラフを描画
plt.figure(figsize=(12, 8))
plt.bar(range(1, n+1), energy_differences, color='blue')
plt.xlabel('Rank of most frequent solution')
plt.ylabel('Energy difference')
plt.title('Energy difference between most frequent solution and its nearest neighbor')
plt.grid(True)
plt.savefig(f'energy_difference_{file_name}_{n}.png')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations

# レスポンスのデータを出現回数でソート
sorted_response = sorted(response.data(), key=lambda x: x[2], reverse=True)

# n番目に多く出現した解を取得
n = 1  # 任意に指定
target_sample = sorted_response[n-1].sample
print(f'{n}th most frequent sample: {target_sample}')

# 異なる量子の数を計算する関数
def count_differing_qubits(sample1, sample2):
    return sum(sample1[q] != sample2[q] for q in sample1)

# 異なる量子の数を格納するリスト
differences = []

# その他の出現した解との比較
for i, res in enumerate(sorted_response):
    if i != n-1:  # n番目に多く出現した解は除外
        sample = res.sample
        diff_count = count_differing_qubits(target_sample, sample)
        differences.append((i+1, diff_count, res.energy, res.num_occurrences))

# データフレームに変換
df = pd.DataFrame(differences, columns=['Rank', 'Differing Qubits', 'Energy', 'Occurrence'])

# Energyの低い順にソート
df = df.sort_values(by='Energy')

# 表を表示
print(df)

# 表を保存
df.to_csv(f'differences_{file_name}_{n}th_most_frequent_sample.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations

# レスポンスのデータを出現回数でソート
sorted_response = sorted(response.data(), key=lambda x: x[2], reverse=True)

# ２番目に出現回数の多い解のエネルギーを取得
num = 1
second_most_frequent_energy = sorted_response[num].energy
print('Second most frequent energy:', second_most_frequent_energy)

# ２番目に出現回数の多い解を取得
second_most_frequent_sample = sorted_response[num].sample

# 異なる値をとる量子の数ごとにエネルギーを格納する辞書
all_energies_by_diff_count = {}

# ２番目に出現回数の多い解のサンプルと異なる値をとる量子が4つ以内のすべての組み合わせを生成
for diff_count in range(1, 5):
    for indices in combinations(range(len(second_most_frequent_sample)), diff_count):
        sample = second_most_frequent_sample.copy()
        for i in indices:
            sample[f'q{i:03}'] = 1 - sample[f'q{i:03}']  # フリップ
        energy = model.energy(sample, vartype='BINARY')
        
        if diff_count not in all_energies_by_diff_count:
            all_energies_by_diff_count[diff_count] = []
        
        all_energies_by_diff_count[diff_count].append(energy)

# 異なる値をとる量子の数ごとにエネルギーを表示
for diff_count, energies in sorted(all_energies_by_diff_count.items()):
    print(f'Number of differing qubits: {diff_count}, Energies: {energies}')

# グラフを描画
plt.figure(figsize=(12, 8))
boxplot_data = [energies for diff_count, energies in sorted(all_energies_by_diff_count.items())]
labels = [str(diff_count) for diff_count in sorted(all_energies_by_diff_count.keys())]
plt.boxplot(boxplot_data, labels=labels)

# 各箱ひげ図の上にエネルギーの件数を表示
for i, energies in enumerate(boxplot_data, start=1):
    count = len(energies)
    plt.text(i, max(energies) + 0.1, f'n={count}', ha='center', va='bottom')

# ２番目に出現回数の多い解のエネルギーをプロット
plt.scatter([0], [second_most_frequent_energy], color='red', zorder=5, label='Second most frequent solution')

# ２番目に出現回数の多い解のエネルギーの横線を引く
plt.axhline(y=second_most_frequent_energy, color='red', linestyle='--', label='Energy of second most frequent solution')

plt.xlabel('Number of differing qubits')
plt.ylabel('Energy')
plt.title('Energy distribution by number of differing qubits')
plt.legend()
plt.grid(True)
plt.savefig(f'energy_distribution_{file_name}_{num}.png')
plt.show()

In [ ]:
import pandas as pd

# 異なる量子の数を計算する関数
def count_differing_qubits(sample1, sample2):
    return sum(sample1[q] != sample2[q] for q in sample1)

# 条件を満たす解を抽出する関数
def extract_solutions(response, num_qubits, model):
    solutions = []
    for target_sample, target_energy in response.data(['sample', 'energy']):
        min_neighbor_energy = float('inf')
        for indices in combinations(range(num_qubits), 1):
            neighbor_sample = target_sample.copy()
            for i in indices:
                key = f'q{i:03}'  # キーを生成
                neighbor_sample[key] = 1 - neighbor_sample[key]  # フリップ
            neighbor_energy = model.energy(neighbor_sample, vartype='BINARY')
            if neighbor_energy < min_neighbor_energy:
                min_neighbor_energy = neighbor_energy
        if target_energy < min_neighbor_energy:
            solutions.append((target_sample, target_energy))
    return solutions

# 条件を満たす解を抽出
solutions = extract_solutions(response_dimod, players, model)

# 抽出した解を表示
for sample, energy in solutions:
    print(f'Sample: {sample}, Energy: {energy}')

# データフレームに変換
df = pd.DataFrame(solutions, columns=['Sample', 'Energy'])

# 表を表示
print(df)

# 表を保存
df.to_csv(f'extracted_solutions_{file_name}.csv', index=False)

In [ ]:
# レスポンスのデータを出現回数でソート
sorted_response = sorted(response.data(), key=lambda x: x[2], reverse=True)
# 量子計算で求められた解をリストに変換
quantum_solutions = [tuple(sample.values()) for sample, energy in response.data(['sample', 'energy'])]

# dfの解をリストに変換
df_solutions = [tuple(sample.values()) for sample in df['Sample']]

# 一致している解を抽出
matching_solutions = [sol for sol in df_solutions if sol in quantum_solutions]

# 一致している解の数を表示
print(f'Number of matching solutions: {len(matching_solutions)}')

# 一致している解の情報を表示
for sol in matching_solutions:
    energy = next(energy for sample, energy in response.data(['sample', 'energy']) if tuple(sample.values()) == sol)
    print(f'Sample: {sol}, Energy: {energy}')